<a href="https://colab.research.google.com/github/universeatmyfeet/NLP/blob/master/M3/m3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import zipfile
import numpy as np
import re
import nltk
from zipfile import ZipFile
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
import string
from sklearn.decomposition import NMF,LatentDirichletAllocation 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from urllib.request import urlopen


In [0]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import io

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
 !wget https://github.com/amsurve/J050_sem6_nlp/blob/master/datasets/10k-german-news-articles.zip?raw=true #getting the dataset link

--2020-03-03 18:13:42--  https://github.com/amsurve/J050_sem6_nlp/blob/master/datasets/10k-german-news-articles.zip?raw=true
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/amsurve/J050_sem6_nlp/raw/master/datasets/10k-german-news-articles.zip [following]
--2020-03-03 18:13:42--  https://github.com/amsurve/J050_sem6_nlp/raw/master/datasets/10k-german-news-articles.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/amsurve/J050_sem6_nlp/master/datasets/10k-german-news-articles.zip [following]
--2020-03-03 18:13:42--  https://media.githubusercontent.com/media/amsurve/J050_sem6_nlp/master/datasets/10k-german-news-articles.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.12

In [0]:
! mkdir newsarticles
! unzip 10k-german-news-articles.zip?raw=true -d newsarticles

mkdir: cannot create directory ‘newsarticles’: File exists
Archive:  10k-german-news-articles.zip?raw=true
replace newsarticles/Annotations.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace newsarticles/Annotations_consolidated.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace newsarticles/Articles.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace newsarticles/Categories.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace newsarticles/CrossValSplit.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace newsarticles/Newspaper_Staff.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace newsarticles/Posts.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:
df = pd.read_csv('/content/newsarticles/Articles.csv')

In [0]:
df.head()

,ID_Article,Path,publishingDate,Title,Body
0,1,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro..."
1,2,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ..."
2,3,Diverses/mobil,2013-11-22 12:15:00.00,Die Android App von derStandard.at,"<div class=""section"" id=""content-main"" itempro..."
3,4,Newsroom/User/mitmachen/Mitreden,2014-08-13 05:30:00.00,Welche Erfahrungen haben Sie als Linkshänder g...,"<div class=""section"" id=""content-main"" itempro..."
4,5,Newsroom/User/mitmachen/Mitreden,2014-08-27 12:27:01.09,Wie haben Sie das Jahr 1989 erlebt?,"<div class=""section"" id=""content-main"" itempro..."


In [0]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words('german'))#german stopwords

In [0]:
df['Title']

0                        Die Newsletter von derStandard.at
1                      Werden Sie Teil von derStandard.at!
2                       Die Android App von derStandard.at
3        Welche Erfahrungen haben Sie als Linkshänder g...
4                      Wie haben Sie das Jahr 1989 erlebt?
                               ...                        
12082            Max Prosa: Junger Troubadour alter Schule
12083           "Können Adblocker nicht einfach hinnehmen"
12084             Die Rechten machen Facebook zum Hatebook
12085    "Wrestling Rita": Feministischer Punktsieg im ...
12086             Leopold-Museum feuert Provenienzforscher
Name: Title, Length: 12087, dtype: object

In [0]:
#### TOPIC MODELING - TITLE COLUMN ##########

def toktoktok(text): #preprocessing function 
  text=text.lower()
  text = text.translate(text.maketrans('', '', string.digits))#translate deletes digits and no other characters will be replaced
  text = text.translate(text.maketrans('', '', '''!()-[]{};:'"\,<>./?@#$%^&*_~'''))#translate deletes punctuality and no other characters will be replaced
  t = word_tokenize(text)#tokenizing text thorugh nltk library
  text = ' '.join([word for word in t if word not in stopwords])#removing stopwords(german)
  return text


In [0]:
df1 = df[:1000]

In [0]:
df1['Titleprocessed'] = df1['Title'].apply(toktoktok)#applying preprocessing function

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
tf = TfidfVectorizer()#Feature extraction using tf-idf

In [0]:
X = tf.fit_transform(df1['Titleprocessed'])#applying transform method of vectorizer class on processed data
dense = X.todense().tolist()# dense representation of tfidf matrix meaning that memory will be allocated to store even the missing values in the dataframe
X = pd.DataFrame(dense, columns = tf.get_feature_names())#columns will be the features with column values denoting tf-idf value for the word in that doc

# Topic Modeling

NMF : 



*   Similar to PCA

*   In a text document, the same word can occur in different places with different meanings. For example, "spread" can be applied to the area or to the verb. NMF introduces context here thus increasing accuracy.
*   NMF finds two matrices h and w such that h*w = original data matrix 
* w (vectors) — the topics discovered from the documents.
  h (Coefficients) — the weights for the topics in each document.
*   NMF then maps the original data into the new set of features discovered by it.





In [0]:
nmf = NMF(n_components=10, init='nndsvd', random_state=0)
##Parameters:
# n_components = 10 selecting 10 features
#‘nndsvd’ if n_components <= min(n_samples, n_features)

In [0]:
w = nmf.fit_transform(X)
print(w)#returns transformed data

[[0.00303513 0.         0.         ... 0.         0.         0.        ]
 [0.00303513 0.         0.         ... 0.         0.         0.        ]
 [0.03938817 0.         0.         ... 0.         0.03024306 0.        ]
 ...
 [0.16194364 0.         0.         ... 0.02591331 0.         0.        ]
 [0.         0.         0.         ... 0.00107252 0.0012105  0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [0]:
h = nmf.components_
print(h)

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 2.39039997e-04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.66902777e-02 0.00000000e+00 2.99686258e-04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 7.35661767e-03 ... 2.15474096e-88
  3.47457941e-03 0.00000000e+00]
 [0.00000000e+00 3.50219318e-85 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.95276390e-86]]


In [0]:
def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(documents[doc_index])

In [0]:
display_topics(h,w,list(X),df1["Titleprocessed"],1,5)

Topic 0:
neue
android m erster blick neue softwaregeneration
neue kämpfe ukraine feuer fronten
neue kämpfe ostukraine tschechows gewehr
neue features livebericht chat
neue schlichtungsstelle enttäuschte reisende
Topic 1:
wien
jahre wien karten
zwischenfälle identitärendemo wien
mythos wohnungsknappheit wien
symbolische flüchtlingsgräber wien
erster bitcoingeldautomat startet wien
Topic 2:
österreich
russland österreich
österreich bein achtelfinale
start arzneimittelversandhandel österreich juni
anschluss österreich verloren
österreich hinkt eu internetgeschwindigkeit
Topic 3:
wiener
wiener spö fpö hände spielen
wiener spö startet antifpöwahlkampf
neue regeln wiener gemeindewohnungen grüne greifen spö
landtagswahl spö fpö koalition burgenland abschließen
beginnt steirischer spö diskussion pakt fpö
Topic 4:
apple
swift apple macht programmiersprache open source
warum taylor swift vs apple niemand gewinnt
warum apple macsparte einstampfen
apple co warum uskonzerne riesige geldreserven hor

# LDA:


In [0]:
lda = LatentDirichletAllocation(n_components = 10, random_state = 42)

In [0]:
lda.fit(X)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [0]:

lda.components_

array([[0.82859167, 0.1       , 0.1       , ..., 0.1       , 0.87085024,
        0.50824828],
       [0.10000001, 0.54721358, 0.1       , ..., 0.1       , 0.1       ,
        0.1       ],
       [0.10000001, 0.1       , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ],
       ...,
       [0.10000001, 0.1       , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ],
       [0.48954803, 0.1       , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ],
       [0.44663875, 0.1       , 0.496509  , ..., 0.80710677, 0.1       ,
        0.1       ]])

In [0]:


def print_topics(model, count_vectorizer, n_top_words):
    topics = []
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        topic = " ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]])
        topics.append(topic)

    return topics
        

In [0]:
topics_title = print_topics(lda, tf, 10)

In [0]:
topics_title

['österreich russland wien verweigert ergebnisse landtagswahlen jahr derstandardat alternative start',
 'android community erneut österreich arbeitet chef leute villa scheitert league',
 'usa mehr macht apple spö neue kostennutzenrechnung lebe gut leben',
 'offtopicforum apple ios zwei große jailbreak wiener schützen tatort festplattenabgabe',
 'wien neue wiener kommt österreich graz one letzte achtelfinale zurück',
 'drei tote jahre ende neue obama frieden wiener terrorprozess androidsmartphones',
 'wiesberger bilderberger gespräche griechenland bringt statt kommen hält google europa',
 'sieht besser fleischkenner koalition prozent linz verletzt jähriger burgenland seit',
 'ggegner diskussion österreicher fürchtet communityforum erwartet rücken privates wochenende protest',
 'neue google fordert robert einigung knapp gibt of österreichs gajeta']

In [0]:
#### TOPIC MODELING - BODY COLUMN ##########

#Removing html tags from Body column 
import re
def remove__htmltags(data):
    return re.sub(r'<.*?>','',data)

In [0]:
df1['Body1'] = df1['Body'].apply(remove__htmltags)
df1['Body1'] = df1['Body1'].apply(toktoktok)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
df1['Body1']

0      abonnieren unserer newsletter lassen aktuelle ...
1      teil derstandardat dezember function popupwinp...
2      smartphoneapp bietet nachrichten echtzeit mögl...
3      sieht alltag linkshänder erledigen arbeiten re...
4      erzählen erlebnissen rund weltpolitischen umbr...
                             ...                        
995    lotterien sollen prozent dach konzernchef stos...
996    microsoftceo nadella kündigt harte entscheidun...
997    hersteller blutigen games anfangen kannst dich...
998    grüner abgeordneter flächendeckendem lauschang...
999    zunächst sicherheitshalber speziell geschulte ...
Name: Body1, Length: 1000, dtype: object

In [0]:
df1.shape

(1000, 7)

In [0]:
tf1 = TfidfVectorizer()

In [0]:
X1 = tf1.fit_transform(df1['Body1'])
dense1 = X1.todense().tolist()
X1 = pd.DataFrame(dense1, columns = tf1.get_feature_names())

In [0]:
nmf = NMF(n_components=5, init='nndsvd', random_state=0, alpha=0.1, l1_ratio=0.5)

In [0]:
w1 = nmf.fit_transform(X1)
print(w1)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [0]:
h1 = nmf.components_
print(h1)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
display_topics(h1,w1,list(X1),df1["Body1"],1,5)

Topic 0:
prozent
prozent eurozone prozent österreichluxemburgbrüssel arbeitslosenrate eurozone april gegenüber vormonat märz leicht zurückgegangen eurostat mittwoch mitteilte betrug rate april euroraum prozent gegenüber prozent märz österreich kommt quote prozent mehr dritten rang eu gemeinsam luxemburg malta deutschland prozent großbritannien prozent allerdings stammen daten london februar neuere vorlagen österreich zuletzt zweitbesten daten eu aufwies verzeichnet rückfall nachdem länder abstand alpenrepublik vergangenen monaten reduzieren konnten märz österreich arbeitslosenrate prozent aufgewiesen april prozent prozent euschnitt gesamten eu betrug arbeitslosenquote april prozent unverändert gegenüber märz höchsten quoten registrierten vormonaten griechenland prozent spanien prozent jahr betrachtet fiel arbeitslosenquote april eustaaten sechs ländern darunter österreich stieg anstiege verzeichneten belgien rumänien frankreich kroatien finnland stärksten rückgänge meldeten spanien lit

In [0]:
lda1 = LatentDirichletAllocation(n_components = 5, random_state = 42)

In [0]:
lda1.fit(X1)
lda1.components_

array([[0.20001096, 0.20001086, 0.20422519, ..., 0.27161914, 0.2000046 ,
        0.20000445],
       [0.20001104, 0.26576216, 0.20000789, ..., 0.20000785, 0.23048132,
        0.35248079],
       [0.31804548, 0.20001078, 0.26176415, ..., 0.20000795, 0.2000046 ,
        0.20000446],
       [0.20001094, 0.20001051, 0.20000796, ..., 0.2000077 , 0.20000461,
        0.20000447],
       [0.20001062, 0.20001128, 0.2000079 , ..., 0.20000772, 0.2000046 ,
        0.20000445]])

In [0]:
topics_body = print_topics(lda1, tf1, 10)

In [0]:
topics_body

['prozent mehr euro sei griechenland wurde österreich sagte millionen seit',
 'prozent google sei euro österreich mehr laut wurde zwei apa',
 'mehr prozent wurde sei spö sagte regierung jahr euro neue',
 'mehr wurde sei prozent apple wien euro zwei österreich sagte',
 'wurde params sporttabellenteaser var mehr sei fpö zwei schon österreich']

## Similarity between topics of title and body

In [0]:
topicsall = topics_body + topics_title #every topic becomes a doc

In [0]:
tf = TfidfVectorizer(min_df=1, stop_words="english")                                                                                                                                                                                                   
tfidf = tf.fit_transform(topicsall)                                                                                                                                                                                                                       
similarity = tfidf * tfidf.T #cos theta = tf matrix dot product tfidf's transpose

In [0]:
topicsall

['prozent mehr euro sei griechenland wurde österreich sagte millionen seit',
 'prozent google sei euro österreich mehr laut wurde zwei apa',
 'mehr prozent wurde sei spö sagte regierung jahr euro neue',
 'mehr wurde sei prozent apple wien euro zwei österreich sagte',
 'wurde params sporttabellenteaser var mehr sei fpö zwei schon österreich',
 'österreich russland wien verweigert ergebnisse landtagswahlen jahr derstandardat alternative start',
 'android community erneut österreich arbeitet chef leute villa scheitert league',
 'usa mehr macht apple spö neue kostennutzenrechnung lebe gut leben',
 'offtopicforum apple ios zwei große jailbreak wiener schützen tatort festplattenabgabe',
 'wien neue wiener kommt österreich graz one letzte achtelfinale zurück',
 'drei tote jahre ende neue obama frieden wiener terrorprozess androidsmartphones',
 'wiesberger bilderberger gespräche griechenland bringt statt kommen hält google europa',
 'sieht besser fleischkenner koalition prozent linz verletzt j

In [0]:
similarity.toarray()  # similarity is a sparse matrix with the number of rows and columns equal to the number of docs

array([[1.        , 0.43512797, 0.48391111, 0.58912112, 0.23927936,
        0.04382169, 0.0422739 , 0.0526649 , 0.        , 0.04902385,
        0.        , 0.10514372, 0.16449192, 0.        , 0.        ],
       [0.43512797, 1.        , 0.37626903, 0.56877801, 0.31938755,
        0.04387985, 0.04233001, 0.05273481, 0.07146737, 0.04908892,
        0.        , 0.08384792, 0.05836019, 0.        , 0.09058804],
       [0.48391111, 0.37626903, 1.        , 0.5242576 , 0.18855821,
        0.10820724, 0.        , 0.22517504, 0.        , 0.06642836,
        0.05819008, 0.        , 0.05770084, 0.        , 0.06171381],
       [0.58912112, 0.56877801, 0.5242576 , 1.        , 0.34555808,
        0.14177865, 0.04579852, 0.15442567, 0.17142245, 0.15860948,
        0.        , 0.        , 0.06314221, 0.        , 0.        ],
       [0.23927936, 0.31938755, 0.18855821, 0.34555808, 1.        ,
        0.03890369, 0.03752961, 0.04675445, 0.06336266, 0.04352202,
        0.        , 0.        , 0.        , 

In [0]:
sim = similarity.toarray()     
np.fill_diagonal(sim, np.nan) # to mask 1- similarity of doc to itself                                                                                                                                                                                                                           

doc = "usa mehr macht apple spö neue kostennutzenrechnung lebe gut leben"                                                                                                                                                                                                  
idx = topicsall.index(doc)  # index of the topic in the docs corpus                                                                                                                                                                                                                   
idx                                                                                                                                                                                                                                                


7

In [0]:
final_idx = np.nanargmax(sim[idx])#argmax of the index row gives the most similar document                                                                                                                                                                                                                
topicsall[final_idx]  # gives the topic most similar to the input topic


'mehr prozent wurde sei spö sagte regierung jahr euro neue'

## Similarity between topics of title and Original Title column



In [0]:
title = df1["Titleprocessed"].to_list()

In [0]:
topic_title_with_Title = title + topics_title # similarity between original title and and topics of title

In [0]:
tf = TfidfVectorizer(min_df=1, stop_words="english")                                                                                                                                                                                                   
tfidf = tf.fit_transform(topic_title_with_Title)                                                                                                                                                                                                                       
similarity  = tfidf * tfidf.T 

In [0]:
similarity.toarray()  

array([[1.        , 0.3950723 , 0.34035691, ..., 0.        , 0.        ,
        0.        ],
       [0.3950723 , 1.        , 0.34035691, ..., 0.        , 0.        ,
        0.        ],
       [0.34035691, 0.34035691, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [0]:

sim = similarity.toarray()     
np.fill_diagonal(sim, np.nan)                                                                                                                                                                                                                           

doc = "newsletter derstandardat"                                                                                                                                                                                                  
idx = topic_title_with_Title.index(doc)                                                                                                                                                                                                                
idx                                                                                                                                                                                                                                                      

0

In [0]:
final_idx = np.nanargmax(sim[idx])                                                                                                                                                                                                                
topic_title_with_Title[final_idx]

'teil derstandardat'

## Similarity between topics of body and Original Title column



In [0]:
topic_title_with_body = title + topics_body # # similarity between original title and and topics of body

In [0]:
tf = TfidfVectorizer(min_df=1, stop_words="english")                                                                                                                                                                                                   
tfidf = tf.fit_transform(topic_title_with_body)                                                                                                                                                                                                                       
similarity = tfidf * tfidf.T 

In [0]:
similarity.toarray()  

array([[1.        , 0.40585979, 0.3477351 , ..., 0.        , 0.        ,
        0.        ],
       [0.40585979, 1.        , 0.3477351 , ..., 0.        , 0.        ,
        0.        ],
       [0.3477351 , 0.3477351 , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.64419176,
        0.27698705],
       [0.        , 0.        , 0.        , ..., 0.64419176, 1.        ,
        0.43679929],
       [0.        , 0.        , 0.        , ..., 0.27698705, 0.43679929,
        1.        ]])

In [0]:
sim = similarity.toarray()     
np.fill_diagonal(sim, np.nan)                                                                                                                                                                                                                            

doc = "wollten kind"                                                                                                                                                                                                  
idx = topic_title_with_body.index(doc)                                                                                                                                                                                                                      
idx       

7

In [0]:
final_idx = np.nanargmax(sim[idx])                                                                                                                                                                                                                
topic_title_with_body[final_idx]

'neubeginn erwachsenen kind'

# Mapping back


In [0]:
import gensim
def abcd(text):

  processed = word_tokenize(text)
  return processed

lalala = df1['Titleprocessed'].apply(abcd)
dic = gensim.corpora.Dictionary(lalala)# number of times a word appears in the tokens of title

In [0]:
lalala[0]

['newsletter', 'derstandardat']

In [0]:
bow = [dic.doc2bow(doc) for doc in lalala] ##For each document create a dictionary having how many words and how many times those words appear

In [0]:
bow[2]

[(0, 1), (3, 1), (4, 1)]

In [0]:
from gensim import corpora, models
tf = models.TfidfModel(bow)
fulltfidf = tf[bow]

In [0]:
lda = gensim.models.LdaMulticore(fulltfidf, num_topics=5, id2word=dic, passes=2, workers=4)
for idx, topic in lda.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.002*"österreich" + 0.002*"offtopicforum" + 0.002*"start" + 0.002*"wiener" + 0.001*"leben" + 0.001*"mehr" + 0.001*"flüchtlinge" + 0.001*"wien" + 0.001*"erhält" + 0.001*"sieht"
Topic: 1 Word: 0.002*"neue" + 0.002*"wiener" + 0.002*"griechenland" + 0.002*"ios" + 0.002*"apple" + 0.002*"–" + 0.001*"orf" + 0.001*"neuer" + 0.001*"derstandardat" + 0.001*"premier"
Topic: 2 Word: 0.002*"wien" + 0.002*"graz" + 0.002*"zurück" + 0.001*"ende" + 0.001*"blatters" + 0.001*"juni" + 0.001*"entscheiden" + 0.001*"bleibt" + 0.001*"gehen" + 0.001*"landtagswahlen"
Topic: 3 Word: 0.002*"zukunft" + 0.002*"macht" + 0.001*"apple" + 0.001*"alter" + 0.001*"erdogan" + 0.001*"wissen" + 0.001*"warum" + 0.001*"facebook" + 0.001*"regierung" + 0.001*"verlängerung"
Topic: 4 Word: 0.002*"wechselt" + 0.002*"neue" + 0.002*"häupl" + 0.001*"tote" + 0.001*"alijew" + 0.001*"vater" + 0.001*"wiener" + 0.001*"politik" + 0.001*"griechenland" + 0.001*"millionen"


In [0]:
topics_title

['österreich russland wien verweigert ergebnisse landtagswahlen jahr derstandardat alternative start',
 'android community erneut österreich arbeitet chef leute villa scheitert league',
 'usa mehr macht apple spö neue kostennutzenrechnung lebe gut leben',
 'offtopicforum apple ios zwei große jailbreak wiener schützen tatort festplattenabgabe',
 'wien neue wiener kommt österreich graz one letzte achtelfinale zurück',
 'drei tote jahre ende neue obama frieden wiener terrorprozess androidsmartphones',
 'wiesberger bilderberger gespräche griechenland bringt statt kommen hält google europa',
 'sieht besser fleischkenner koalition prozent linz verletzt jähriger burgenland seit',
 'ggegner diskussion österreicher fürchtet communityforum erwartet rücken privates wochenende protest',
 'neue google fordert robert einigung knapp gibt of österreichs gajeta']

In [0]:
#After training LDA topic model , input documents into the model and it will classify them into the relevant topic no
input_vec = dic.doc2bow(lalala[0])
topic_vec = lda[input_vec]

In [0]:
topic_vec

[(0, 0.72766477),
 (1, 0.070362486),
 (2, 0.06669048),
 (3, 0.068589665),
 (4, 0.0666926)]

In [0]:
words = lda.show_topic(0, topn = 200)#top 200 words that make up that topic number.

In [0]:
words

[('österreich', 0.0022387241),
 ('offtopicforum', 0.00202348),
 ('start', 0.0018127387),
 ('wiener', 0.0016033993),
 ('leben', 0.0014032791),
 ('mehr', 0.0013828004),
 ('flüchtlinge', 0.0013594758),
 ('wien', 0.0013317298),
 ('erhält', 0.0012794819),
 ('sieht', 0.0012565382),
 ('eu', 0.0012535155),
 ('einigung', 0.001249867),
 ('neuer', 0.0012378173),
 ('russland', 0.0012307373),
 ('community', 0.0012267102),
 ('geht', 0.0012181646),
 ('–', 0.0011838971),
 ('ab', 0.0011693358),
 ('drei', 0.0011551679),
 ('macht', 0.0011490851),
 ('jemengespräche', 0.0011401448),
 ('linzer', 0.0011276059),
 ('genf', 0.0011122321),
 ('zweifel', 0.0011048225),
 ('communityforum', 0.0011016018),
 ('kostennutzenrechnung', 0.0011014083),
 ('wochen', 0.0010993306),
 ('griechenlandkrise', 0.0010757272),
 ('kommt', 0.0010713271),
 ('regenbogenparade', 0.0010546221),
 ('schaffen', 0.0010520905),
 ('paar', 0.0010356142),
 ('spö', 0.0010333003),
 ('stärker', 0.0010140736),
 ('fpö', 0.00101006),
 ('video', 0.001005

## PyLDA vis

In [0]:
!pip install pyLDAvis
import pickle

tokens = df1["Titleprocessed"].apply(nltk.word_tokenize)

dic = corpora.Dictionary(tokens) 
corp = [dic.doc2bow(t) for t in tokens]

pickle.dump(corp, open('corpus.pkl', 'wb'))
dic.save('dictionary.gensim')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
#LDA using gensim
lda = gensim.models.ldamodel.LdaModel(corp, num_topics = 10 , id2word=dic, passes=15,alpha = 0.1)
lda.save('model5.gensim')
topics = lda.print_topics(num_words=20)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
topics

[(0,
  '0.010*"österreich" + 0.007*"apple" + 0.005*"athen" + 0.005*"faymann" + 0.004*"warum" + 0.004*"neuer" + 0.004*"mehr" + 0.004*"–" + 0.004*"immer" + 0.004*"neue" + 0.004*"südkorea" + 0.002*"geht" + 0.002*"österreicher" + 0.002*"ende" + 0.002*"europa" + 0.002*"facebook" + 0.002*"millionen" + 0.002*"zeigt" + 0.002*"verfügbar" + 0.002*"gibt"'),
 (1,
  '0.004*"wien" + 0.004*"kauft" + 0.003*"eu" + 0.003*"zukunft" + 0.003*"wechselt" + 0.003*"startet" + 0.003*"getötet" + 0.003*"macht" + 0.003*"mexiko" + 0.003*"chef" + 0.003*"world" + 0.003*"südafrika" + 0.003*"erdogan" + 0.003*"drei" + 0.003*"gewinnen" + 0.003*"halten" + 0.003*"vs" + 0.003*"kunst" + 0.003*"fair" + 0.003*"monate"'),
 (2,
  '0.006*"spö" + 0.004*"fpö" + 0.004*"deutsche" + 0.003*"geht" + 0.003*"macht" + 0.003*"forschung" + 0.003*"ametsreiter" + 0.003*"omv" + 0.003*"mal" + 0.003*"bringt" + 0.003*"diskussion" + 0.003*"googles" + 0.003*"jemen" + 0.003*"ablinger" + 0.003*"apple" + 0.003*"neue" + 0.001*"regierung" + 0.001*"usa" +

In [0]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
